<a href="https://colab.research.google.com/github/jvinodraj/theEagle/blob/master/notebooks/KM_Split_by_avgpwr_HR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Upload the FIT file
from google.colab import files
uploaded = files.upload()

# Step 2: Read the uploaded file
from fitparse import FitFile
import pandas as pd

# Get the uploaded file name
filepath = list(uploaded.keys())[0]

def read_fit_file(filepath):
    fitfile = FitFile(filepath)

    records = []
    for record in fitfile.get_messages('record'):
        data = {}
        for field in record:
            data[field.name] = field.value
        records.append(data)

    df = pd.DataFrame(records)

    # Filter only necessary columns
    df = df[['timestamp', 'distance', 'heart_rate', 'power']]
    df = df.dropna()

    # Distance comes in meters; create 'km' column
    df['km'] = (df['distance'] / 1000).astype(int) + 1  # 1-indexed

    return df

def split_by_km(df):
    km_split = df.groupby('km').agg(
        avg_power=('power', 'mean'),
        avg_hr=('heart_rate', 'mean')
    ).reset_index()

    km_split['avg_power'] = km_split['avg_power'].round(1)
    km_split['avg_hr'] = km_split['avg_hr'].round(1)

    return km_split

# Step 3: Process the data
df = read_fit_file(filepath)
km_stats = split_by_km(df)

# Step 4: Show the results
print(km_stats)

# Optional: Save to CSV
km_stats.to_csv('run_split_by_km.csv', index=False)

# Optional: Download the CSV
#files.download('run_split_by_km.csv')
